<a href="https://colab.research.google.com/github/myrah/AAI2025/blob/dev/Agent/routerApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#Install prerequisite packages
#!pip install python-dotenv

!pip install llama-index
!pip install llama-index-llms-openai
!pip install llama-index-embeddings-openai
#!pip install --upgrade llama-index llama-index-llms-openai

In [12]:
# Setup OpenAI connection
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
import os
import nest_asyncio
# from openai import OpenAI # Removed conflicting import
from google.colab import userdata

nest_asyncio.apply()

# Set your OpenAI API key (from https://platform.openai.com/api-keys)
api_key=userdata.get("OPENAI_EDU_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key  # optional but recommended

# Configure LlamaIndex to use OpenAI models
Settings.llm = OpenAI()
Settings.llm.model = "gpt-4-turbo"  # Using a more capable model for the router


Settings.embed_model = OpenAIEmbedding(api_key=api_key, model="text-embedding-3-small")  # Pass api_key explicitly

In [9]:
#Create indexes for vector search
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import  VectorStoreIndex

splitter=SentenceSplitter(chunk_size=1024)

#-------------------------------------------------------------------
#Setup Aeroflow document index
#-------------------------------------------------------------------
aeroflow_documents=SimpleDirectoryReader(
    input_files=["AeroFlow_Specification_Document.pdf"])\
            .load_data()

#Read documents into nodes
aeroflow_nodes=splitter.get_nodes_from_documents(aeroflow_documents)
#Create a vector Store
aeroflow_index=VectorStoreIndex(aeroflow_nodes)
#Create a query engine
aeroflow_query_engine = aeroflow_index.as_query_engine()

#-------------------------------------------------------------------
#Setup EchoSprint document index
#-------------------------------------------------------------------
ecosprint_documents=SimpleDirectoryReader(
    input_files=["EcoSprint_Specification_Document.pdf"])\
            .load_data()
#Read documents into nodes
ecosprint_nodes=splitter.get_nodes_from_documents(ecosprint_documents)
#Create a vector Store
ecosprint_index=VectorStoreIndex(ecosprint_nodes)
#Create a query engine
ecosprint_query_engine = ecosprint_index.as_query_engine()

In [14]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

#Create a query engine Tool for NoSQL
aeroflow_tool = QueryEngineTool.from_defaults(
    query_engine=aeroflow_query_engine,
    name="Aeroflow specifications",
    description=(
        "Contains comprehensive information about the Aeroflow product, including its design, technical features, colors, dimensions, performance specifications, maintenance procedures, and warranty details."
    ),
)

#Create a query engine Tool for NLP
ecosprint_tool = QueryEngineTool.from_defaults(
    query_engine=ecosprint_query_engine,
    name="EcoSprint specifications",
    description=(
        "Contains comprehensive information about the EcoSprint product, including its design, technical features, colors, dimensions, performance specifications, maintenance procedures, and warranty details."
    ),
)

#Create a Router Agent. Provide the Tools to the Agent
router_agent=RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        aeroflow_tool,
        ecosprint_tool,
    ],
    verbose=True
)

In [15]:
#Ask a question about NoSQL
response = router_agent.query("What colors are available for AeroFlow?")
print("\nResponse: ",str(response))

Selecting query engine 0: The first choice specifically mentions comprehensive information about the Aeroflow product, including its colors, which directly addresses the question about what colors are available for Aeroflow..

Response:  The AeroFlow is available in Coastal Blue, Sunset Orange, and Pearl White.


In [16]:
response = router_agent.query("What colors are available for EcoSprint?")
print("\nResponse: ",str(response))

Selecting query engine 1: This choice specifically mentions comprehensive information about the EcoSprint product, including its colors, making it directly relevant to the question about what colors are available for EcoSprint..

Response:  The EcoSprint is available in Midnight Black, Ocean Blue, and Pearl White.
